<a href="https://colab.research.google.com/github/yang1325/AI_selfpraticenote/blob/main/tensorflow_covid/tensorflow_Coviddata_splitted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#載入所需模組
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
import numpy as np
import pandas as pd
from numpy import load
from numpy import save
import random
import os
import gc

In [8]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(299,299, 1)))
cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Conv2D(512, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
#cnn_model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
#cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Flatten())
cnn_model.add(tf.keras.layers.Dropout(0.3))
cnn_model.add(layers.Dense(512, activation='relu'))
cnn_model.add(layers.Dense(4, activation="softmax"))
cnn_model.summary()
gc.collect()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 297, 297, 128)     1280      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 99, 99, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 97, 97, 256)       295168    
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 32, 32, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 30, 30, 512)       1180160   
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 10, 10, 512)     

1290

In [ ]:
path = "/content/gdrive/My Drive/splited_data"

cnn_model.compile(optimizer="Adam",
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

for i in range(100):
  for j in range(10):
    print("now is %d,%d"%(i,j))
    data = load(path + "/%d"%(j) + ".npy")
    label = load(path + "/%dy"%(j) + ".npy")
    if(j==9):
      print(cnn_model.evaluate(data, label, verbose=2))
    else:
      cnn_model.fit(data, label, epochs=1,batch_size = 1)
    del data
    del label
    gc.collect()

now is 0,0
2116/2116 [==============================] - 36s 16ms/step - loss: 1.1672 - accuracy: 0.4915
now is 0,1
2116/2116 [==============================] - 35s 16ms/step - loss: 0.9630 - accuracy: 0.6054
now is 0,2
2116/2116 [==============================] - 35s 17ms/step - loss: 0.7817 - accuracy: 0.6810
now is 0,3
2116/2116 [==============================] - 35s 17ms/step - loss: 0.7259 - accuracy: 0.7141
now is 0,4
2116/2116 [==============================] - 35s 16ms/step - loss: 0.6606 - accuracy: 0.7429
now is 0,5
2116/2116 [==============================] - 35s 16ms/step - loss: 0.5989 - accuracy: 0.7491
now is 0,6
2116/2116 [==============================] - 35s 17ms/step - loss: 0.5911 - accuracy: 0.7736
now is 0,7
2116/2116 [==============================] - 35s 17ms/step - loss: 0.5848 - accuracy: 0.7628
now is 0,8
2116/2116 [==============================] - 35s 16ms/step - loss: 0.5491 - accuracy: 0.7807
now is 0,9
67/67 - 8s - loss: 0.5174 - accuracy: 0.8010 - 8s/epo

In [14]:
#data
print(cnn_model.evaluate(data, train_label, verbose=2))

67/67 - 6s - loss: 1.1753 - accuracy: 0.4948 - 6s/epoch - 92ms/step
[1.17534601688385, 0.49480152130126953]


In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()